In [1]:
import pandas as pd

# Define file paths
file_paths = {
    "Nutrient_Values": "2021-2023 FNDDS At A Glance - FNDDS Nutrient Values.xlsx",
    "Ingredient_Nutrient_Values": "2021-2023 FNDDS At A Glance - Ingredient Nutrient Values.xlsx",
    "Portions_and_Weights": "2021-2023 FNDDS At A Glance - Portions and Weights.xlsx",
    "Ingredients": "2021-2023 FNDDS At A Glance - FNDDS Ingredients.xlsx",
    "Foods_and_Beverages": "2021-2023 FNDDS At A Glance - Foods and Beverages.xlsx"
}

# Output file path
output_file = "Merged_FNDDS_Dataset.xlsx"

# Initialize a writer for the Excel file
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet_name, file_path in file_paths.items():
        # Read each Excel file into a DataFrame
        df = pd.read_excel(file_path)
        # Write the DataFrame to a new sheet in the output Excel file
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Merged Excel file has been saved to {output_file}.")


Merged Excel file has been saved to Merged_FNDDS_Dataset.xlsx.


In [14]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Load the merged dataset
merged_file = "Merged_FNDDS_Dataset.xlsx"
nutrient_values = pd.read_excel(merged_file, sheet_name="Nutrient_Values")

# Define macronutrient targets per meal
total_calories = 1600
total_protein = 100
total_fat = 70
total_carbs = 250

meal_targets = {
    "Breakfast": {"calories": total_calories * 0.3, "protein": total_protein * 0.3, "fat": total_fat * 0.3, "carbs": total_carbs * 0.3},
    "Lunch": {"calories": total_calories * 0.4, "protein": total_protein * 0.4, "fat": total_fat * 0.4, "carbs": total_carbs * 0.4},
    "Dinner": {"calories": total_calories * 0.3, "protein": total_protein * 0.3, "fat": total_fat * 0.3, "carbs": total_carbs * 0.3},
}

# Extract relevant columns for optimization
foods = nutrient_values[["Main food description", "Energy (kcal)", "Protein (g)", "Total Fat (g)", "Carbohydrate (g)"]]
foods = foods.dropna()  # Drop rows with missing values

# Objective function to minimize for one food item
def objective(x, food_index, meal_targets):
    quantity = x[0]
    total_calories = quantity * foods.loc[food_index, "Energy (kcal)"] / 100
    total_protein = quantity * foods.loc[food_index, "Protein (g)"] / 100
    total_fat = quantity * foods.loc[food_index, "Total Fat (g)"] / 100
    total_carbs = quantity * foods.loc[food_index, "Carbohydrate (g)"] / 100
    
    return (
        (total_calories - meal_targets["calories"]) ** 2 +
        (total_protein - meal_targets["protein"]) ** 2 +
        (total_fat - meal_targets["fat"]) ** 2 +
        (total_carbs - meal_targets["carbs"]) ** 2
    )

# Run optimization for a single food item
def optimize_food_for_meal(meal_targets, excluded_foods):
    best_food = None
    best_quantity = None
    best_score = float("inf")
    
    for food_index in range(len(foods)):
        food_name = foods.iloc[food_index]["Main food description"]
        if food_name in excluded_foods:
            continue  # Skip already selected foods
        
        # Bounds for quantity: 0 to 500 grams
        bounds = [(0, 500)]
        
        # Initial guess: 100 grams
        initial_guess = [100]
        
        # Minimize the objective function
        result = minimize(
            objective, initial_guess, args=(food_index, meal_targets), bounds=bounds, method="SLSQP"
        )
        
        # Check if the result is better than the current best
        if result.success and result.fun < best_score:
            best_food = foods.iloc[food_index]
            best_quantity = result.x[0]
            best_score = result.fun
    
    return best_food, best_quantity

# Optimize for each meal
optimized_meals = {}
excluded_foods = set()
for meal, targets in meal_targets.items():
    print(f"Optimizing {meal}...")
    best_food, best_quantity = optimize_food_for_meal(targets, excluded_foods)
    if best_food is not None:
        optimized_meals[meal] = {
            "food_name": best_food["Main food description"],
            "quantity": best_quantity,
            "energy": best_food["Energy (kcal)"] * best_quantity / 100,
            "protein": best_food["Protein (g)"] * best_quantity / 100,
            "fat": best_food["Total Fat (g)"] * best_quantity / 100,
            "carbs": best_food["Carbohydrate (g)"] * best_quantity / 100,
        }
        excluded_foods.add(best_food["Main food description"])  # Add the selected food to the exclusion set

# Display optimized meals
print("\nOptimized Meal Plan:")
for meal, details in optimized_meals.items():
    print(f"\n{meal}:")
    print(f"  Food: {details['food_name']}")
    print(f"  Quantity: {details['quantity']:.2f} g")
    print(f"  Energy: {details['energy']:.2f} kcal")
    print(f"  Protein: {details['protein']:.2f} g")
    print(f"  Fat: {details['fat']:.2f} g")
    print(f"  Carbs: {details['carbs']:.2f} g")


Optimizing Breakfast...
Optimizing Lunch...
Optimizing Dinner...

Optimized Meal Plan:

Breakfast:
  Food: Vegetable mixture, dried
  Quantity: 146.61 g
  Energy: 480.88 kcal
  Protein: 25.52 g
  Fat: 13.02 g
  Carbs: 72.16 g

Lunch:
  Food: Chickpeas, from canned, no added fat
  Quantity: 438.88 g
  Energy: 640.76 kcal
  Protein: 35.94 g
  Fat: 14.22 g
  Carbs: 98.53 g

Dinner:
  Food: Pasta with sauce and meat, from school lunch
  Quantity: 411.07 g
  Energy: 480.96 kcal
  Protein: 24.46 g
  Fat: 12.83 g
  Carbs: 71.94 g


In [1]:
import pandas as pd

# Load the Excel file
mealdb_file = "meals_with_openfoodfacts.xlsx"
meals_df = pd.read_excel(mealdb_file)

# Print the existing columns in the dataset
print("Columns in the dataset:")
print(meals_df.columns)

# Load OpenFoodFacts data for reference
openfoodfact_file = "openfoodfacts_cleaned.csv"
openfoodfacts_data = pd.read_csv(openfoodfact_file)

# Print the OpenFoodFacts columns
print("\nColumns in OpenFoodFacts data:")
print(openfoodfacts_data.columns)


Columns in the dataset:
Index(['Meal Name', 'Ingredients (MealDB)', 'Cross-Referenced',
       'Ingredients (OpenFoodFacts)', 'OpenFoodFacts Codes',
       'Nutrition Facts'],
      dtype='object')


C:\Users\pc\AppData\Local\Temp\ipykernel_40220\232636883.py:13: DtypeWarning: Columns (0,11,14,15,16,17,23,24,25,26,27,31,32,33,34,35,36,37,45,50,51,52,53,56,67,72) have mixed types. Specify dtype option on import or set low_memory=False.
  openfoodfacts_data = pd.read_csv(openfoodfact_file)



Columns in OpenFoodFacts data:
Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'last_modified_by',
       'last_updated_t', 'last_updated_datetime',
       ...
       'glycemic-index_100g', 'water-hardness_100g', 'choline_100g',
       'phylloquinone_100g', 'beta-glucan_100g', 'inositol_100g',
       'carnitine_100g', 'sulphate_100g', 'nitrate_100g', 'acidity_100g'],
      dtype='object', length=206)


In [1]:

mealdb_file = "meals_with_openfoodfacts.xlsx"
openfoodfact_file = "openfoodfacts_cleaned.csv"
output_file = "meals_with_combined_nutrition.xlsx"

# Identify numerical columns in OpenFoodFacts dataset
numerical_columns = openfoodfacts_data.select_dtypes(include=[float, int]).columns.tolist()

# Initialize new columns in the meals dataset
for col in numerical_columns:
    meals_df[col] = 0.0

# Process each meal
for index, meal in meals_df.iterrows():
    ingredients = eval(meal["Ingredients (OpenFoodFacts)"])  # List of OpenFoodFacts ingredient names
    if not isinstance(ingredients, list):  # Skip invalid entries
        continue

    # Filter OpenFoodFacts data for the matching ingredients
    matched_data = openfoodfacts_data[openfoodfacts_data["product_name"].isin(ingredients)]

    # Sum the values for numerical columns and add them to the meal
    for col in numerical_columns:
        if col in matched_data:
            meals_df.at[index, col] = matched_data[col].sum()

# Save the updated dataset
meals_df.to_excel(output_file, index=False)
print(f"Updated dataset saved to {output_file}.")

NameError: name 'openfoodfacts_data' is not defined

In [2]:
import pandas as pd
import numpy as np

# Load the combined meal dataset
combined_file = "meals_with_combined_nutrition.xlsx"
meals_df = pd.read_excel(combined_file)

# Ensure the health score column exists or create one
if "Health Score" not in meals_df.columns:
    def calculate_health_score(row):
        penalties = (
            row.get("sugars_100g", 0) * 2 +
            row.get("saturated-fat_100g", 0) * 3 +
            row.get("sodium_100g", 0) / 1000
        )
        rewards = (
            row.get("fiber_100g", 0) * 2 +
            row.get("proteins_100g", 0) * 1.5 +
            row.get("vitamin-c_100g", 0)
        )
        return rewards - penalties

    meals_df["Health Score"] = meals_df.apply(calculate_health_score, axis=1)
    min_score = meals_df["Health Score"].min()
    max_score = meals_df["Health Score"].max()
    meals_df["Health Score"] = (meals_df["Health Score"] - min_score) / (max_score - min_score) * 100

# User-defined meal preferences
total_calories = float(input("Enter your total daily calorie target: "))
total_protein = float(input("Enter your total daily protein target (g): "))
total_fat = float(input("Enter your total daily fat target (g): "))
total_carbs = float(input("Enter your total daily carbohydrate target (g): "))
num_breakfast = int(input("Enter the number of breakfast items: "))
num_lunch = int(input("Enter the number of lunch items: "))
num_dinner = int(input("Enter the number of dinner items: "))

# Define meal targets
meal_targets = {
    "Breakfast": {"calories": total_calories * 0.3 / num_breakfast, "protein": total_protein * 0.3 / num_breakfast, "fat": total_fat * 0.3 / num_breakfast, "carbs": total_carbs * 0.3 / num_breakfast},
    "Lunch": {"calories": total_calories * 0.4 / num_lunch, "protein": total_protein * 0.4 / num_lunch, "fat": total_fat * 0.4 / num_lunch, "carbs": total_carbs * 0.4 / num_lunch},
    "Dinner": {"calories": total_calories * 0.3 / num_dinner, "protein": total_protein * 0.3 / num_dinner, "fat": total_fat * 0.3 / num_dinner, "carbs": total_carbs * 0.3 / num_dinner},
}

# Save original input targets
original_targets = {
    "calories": total_calories,
    "protein": total_protein,
    "fat": total_fat,
    "carbs": total_carbs,
}

# Realistic serving sizes
serving_fractions = [1, 0.5, 0.33, 0.25]

# Function to optimize single food per meal
def optimize_food_for_meal(meal_targets, excluded_foods):
    best_food = None
    best_score = float("inf")
    best_serving = None

    for index, food in meals_df.iterrows():
        if food["Meal Name"] in excluded_foods:
            continue

        for fraction in serving_fractions:
            quantity = fraction * 100  # Scale to 100g servings
            total_calories = quantity * food["energy-kcal_100g"] / 100
            total_protein = quantity * food["proteins_100g"] / 100
            total_fat = quantity * food["fat_100g"] / 100
            total_carbs = quantity * food["carbohydrates_100g"] / 100

            deviation = (
                (total_calories - meal_targets["calories"]) ** 2 +
                (total_protein - meal_targets["protein"]) ** 2 +
                (total_fat - meal_targets["fat"]) ** 2 +
                (total_carbs - meal_targets["carbs"]) ** 2
            )
            score = deviation - food["Health Score"]

            if score < best_score:
                best_score = score
                best_food = food
                best_serving = fraction

    return best_food, best_serving

# Optimize meals
optimized_meals = {}
excluded_foods = set()
overall_totals = {"calories": 0, "protein": 0, "fat": 0, "carbs": 0}

for meal_name, targets in meal_targets.items():
    print(f"\nOptimizing {meal_name}...")
    optimized_meals[meal_name] = []
    for _ in range(num_breakfast if meal_name == "Breakfast" else (num_lunch if meal_name == "Lunch" else num_dinner)):
        best_food, best_serving = optimize_food_for_meal(targets, excluded_foods)
        if best_food is not None:
            total_calories = best_food["energy-kcal_100g"] * best_serving
            total_protein = best_food["proteins_100g"] * best_serving
            total_fat = best_food["fat_100g"] * best_serving
            total_carbs = best_food["carbohydrates_100g"] * best_serving

            # Update overall totals
            overall_totals["calories"] += total_calories
            overall_totals["protein"] += total_protein
            overall_totals["fat"] += total_fat
            overall_totals["carbs"] += total_carbs

            optimized_meals[meal_name].append({
                "food_name": best_food["Meal Name"],
                "serving": best_serving,
                "quantity": best_serving * 100,
                "energy": total_calories,
                "protein": total_protein,
                "fat": total_fat,
                "carbs": total_carbs,
                "ingredients": best_food["Ingredients (MealDB)"]
            })
            excluded_foods.add(best_food["Meal Name"])

# Calculate match percentages using original targets
calories_match = max(0, 100 - abs(overall_totals["calories"] - original_targets["calories"]) / original_targets["calories"] * 100)
protein_match = max(0, 100 - abs(overall_totals["protein"] - original_targets["protein"]) / original_targets["protein"] * 100)
fat_match = max(0, 100 - abs(overall_totals["fat"] - original_targets["fat"]) / original_targets["fat"] * 100)
carbs_match = max(0, 100 - abs(overall_totals["carbs"] - original_targets["carbs"]) / original_targets["carbs"] * 100)

overall_match_percentage = (calories_match + protein_match + fat_match + carbs_match) / 4

# Print results
for meal_name, foods in optimized_meals.items():
    print(f"\n{meal_name}:")
    for food in foods:
        print(f"  Food: {food['food_name']}")
        print(f"  Ingredients: {food['ingredients']}")
        print(f"  Serving: {food['serving']}x")
        print(f"  Quantity: {food['quantity']:.2f} g")
        print(f"  Energy: {food['energy']:.2f} kcal")
        print(f"  Protein: {food['protein']:.2f} g")
        print(f"  Fat: {food['fat']:.2f} g")
        print(f"  Carbs: {food['carbs']:.2f} g")

print("\nOverall Totals:")
print(f"  Calories: {overall_totals['calories']:.2f} kcal")
print(f"  Protein: {overall_totals['protein']:.2f} g")
print(f"  Fat: {overall_totals['fat']:.2f} g")
print(f"  Carbs: {overall_totals['carbs']:.2f} g")


# Print the overall match percentage
print(f"\nOverall Match Percentage: {overall_match_percentage:.2f}%")


Optimizing Breakfast...

Optimizing Lunch...

Optimizing Dinner...

Breakfast:
  Food: Split Pea Soup
  Ingredients: ['Ham', 'Peas', 'Onions', 'Carrots', 'Bay Leaves', 'Celery', 'Frozen Peas', 'Bread']
  Serving: 0.25x
  Quantity: 25.00 g
  Energy: 306.60 kcal
  Protein: 17.62 g
  Fat: 7.35 g
  Carbs: 18.03 g
  Food: Mbuzi Choma (Roasted Goat)
  Ingredients: ['Goat Meat', 'Corn Flour', 'Tomatoes', 'Salt', 'Onion', 'Green Chilli', 'Coriander Leaves']
  Serving: 0.25x
  Quantity: 25.00 g
  Energy: 308.23 kcal
  Protein: 10.09 g
  Fat: 10.96 g
  Carbs: 37.94 g

Lunch:
  Food: Peanut Butter Cookies
  Ingredients: ['Peanut Butter', 'Sugar', 'Egg']
  Serving: 0.25x
  Quantity: 25.00 g
  Energy: 384.25 kcal
  Protein: 28.10 g
  Fat: 9.83 g
  Carbs: 51.22 g
  Food: Chicken Congee
  Ingredients: ['Chicken', 'Salt', 'Pepper', 'Ginger Cordial', 'Ginger', 'Spring Onions', 'Rice', 'Water', 'Coriander']
  Serving: 0.25x
  Quantity: 25.00 g
  Energy: 409.53 kcal
  Protein: 15.41 g
  Fat: 17.72 g
  C